In [257]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn import metrics

import joblib

train = pd.read_csv('static/data/train.csv')
test = pd.read_csv('static/data/test.csv')

In [258]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [259]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [260]:
y_train = train['label']
X_train = train.drop(columns=['label'])

In [261]:
del train

y_train.value_counts()

label
1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: count, dtype: int64

In [262]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
model = RandomForestClassifier()

In [263]:
print('Number of Trees used : ', model.n_estimators)

Number of Trees used :  100


In [264]:
model.fit(X_train,y_train)

RandomForestClassifier()

In [265]:
predict= model.predict(X_train)
predict

array([1, 0, 1, ..., 7, 6, 9])

In [266]:
cm= metrics.confusion_matrix(y_train,model.predict(X_train))
cm

array([[4132,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 4684,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0, 4177,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0, 4351,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0, 4072,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0, 3795,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0, 4137,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0, 4401,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0, 4063,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0, 4188]])

In [267]:
from sklearn.metrics import accuracy_score
trainaccuracy= accuracy_score(y_train,model.predict(X_train))
trainaccuracy
print("Train Data Accuracy    :{} %".format(round((trainaccuracy*100),2)))

Train Data Accuracy    :100.0 %


In [268]:
from sklearn.model_selection import GridSearchCV

In [269]:
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1)


In [270]:
# Create the parameter grid based on the results of random search 
params = {
    'max_depth': [1, 2, 20],
    'min_samples_leaf': [5, 10],
    'max_features': [2,3,4],
    'n_estimators': [10, 200]
}

In [271]:
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=classifier_rf, param_grid=params, 
                          cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

In [272]:
%%time
grid_search.fit(X_train,y_train)

Fitting 4 folds for each of 36 candidates, totalling 144 fits


CPU times: user 11.5 s, sys: 1.09 s, total: 12.6 s
Wall time: 2min 13s


GridSearchCV(cv=4, estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [1, 2, 20], 'max_features': [2, 3, 4],
                         'min_samples_leaf': [5, 10],
                         'n_estimators': [10, 200]},
             scoring='accuracy', verbose=1)

In [273]:
rf_best = grid_search.best_estimator_
rf_best

RandomForestClassifier(max_depth=20, max_features=4, min_samples_leaf=5,
                       n_estimators=200, n_jobs=-1, random_state=42)

In [274]:
cm= metrics.confusion_matrix(y_train,rf_best.predict(X_train))
cm

array([[4087,    0,    0,    1,    4,    3,   22,    1,   12,    2],
       [   0, 4642,   14,    7,    5,    1,    7,    2,    4,    2],
       [  11,   14, 4081,    7,   16,    0,    2,   27,   17,    2],
       [   3,   23,   33, 4192,    0,    5,    7,   30,   32,   26],
       [   1,   16,    6,    0, 3959,    0,   14,    7,    2,   67],
       [   8,    8,    3,   32,    4, 3698,   29,    1,    4,    8],
       [  13,   10,    1,    0,    3,   12, 4092,    0,    6,    0],
       [   2,   29,   26,    1,   11,    0,    0, 4284,    2,   46],
       [  12,   37,   10,   26,   12,    5,   12,    5, 3922,   22],
       [  16,   17,    4,   60,   19,    1,    2,   47,   19, 4003]])

In [275]:
trainaccuracy= accuracy_score(y_train,rf_best.predict(X_train))
trainaccuracy
print("Train Data Accuracy    :{} %".format(round((trainaccuracy*100),2)))

Train Data Accuracy    :97.52 %


In [276]:
result = model.predict(test)


result = pd.Series(result,name="Label")
result

0        2
1        0
2        9
3        9
4        3
        ..
27995    9
27996    7
27997    3
27998    9
27999    2
Name: Label, Length: 28000, dtype: int64

In [277]:
from PIL import Image

image_file = Image.open('static/img/2.png')

In [291]:
from PIL import Image

image_file = Image.open('static/img/0.png')
image_file = image_file.convert('L')
image_file = image_file.point(lambda x: 255 - x)
image_file = image_file.resize((28, 28))
array_image_file = np.array(image_file)  
array_image_file = array_image_file.flatten() 

predict = model.predict(array_image_file.reshape(1, -1))
predict

/Users/florianberthelot/.pyenv/versions/3.10.7/envs/flask/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0])

In [292]:
path = 'static/model.joblib'

joblib.dump(model, path)

['static/model.joblib']